In [23]:
import pandas as pd
from pprint import pprint

In [24]:
def dim_staff_dataframe(**dataframes):
    """
    Constructs the staff dimension table by merging staff data with department names.

    This function expects DataFrames for 'staff' and 'department' only.
    It merges staff with their corresponding departments and selects key fields
    for OLAP-friendly dimension usage.

    Parameters:
    -----------
    **dataframes : dict
        A dictionary of named DataFrames, expected to contain:
        - 'staff': DataFrame with at least 'staff_id', 'first_name', 'last_name', 'department_id', 'email_address'
        - 'department': DataFrame with 'department_id', 'department_name' and 'location'

    Returns:
    --------
    pd.DataFrame
        A dimension-style DataFrame containing:
        - 'staff_id'
        - 'first_name'
        - 'last_name'
        - 'department_name'
        - 'email_address'

    Raises:
    -------
    ValueError
        If any of the required dataframes are missing from input.
    Exception
        For any unexpected error encountered during merging or transformation.
    """
    required_keys = ["staff", "department"]

    for key in required_keys:
        if key not in dataframes:
            raise ValueError(f"Error: Missing required dataframe '{key}'.")

    staff_df = dataframes.get("staff")
    departments_df = dataframes.get("department")

    try:
        staff_df = staff_df.merge(
            departments_df[["department_id", "department_name", "location"]],
            how="left",
            on="department_id",
        )

        dim_staff = staff_df[
            [
                "staff_id",
                "first_name",
                "last_name",
                "department_name",
                "location",
                "email_address",
            ]
        ].drop_duplicates()

        return dim_staff

    except Exception as e:
        raise Exception(f"Error creating dim_staff: {e}")

In [25]:
staff_df = pd.read_parquet(
    "../sql_local_tests/seed_data/staff_2022-11-3_14-20-51_563000.parquet"
)
department_df = pd.read_parquet(
    "../sql_local_tests/seed_data/department_2022-11-3_14-20-49_962000.parquet"
)
pprint(staff_df.head(3))
pprint(department_df.head(3))

   staff_id first_name last_name  department_id  \
0         1    Jeremie    Franey              2   
1         2      Deron     Beier              6   
2         3   Jeanette    Erdman              6   

                       email_address              created_at  \
0   jeremie.franey@terrifictotes.com 2022-11-03 14:20:51.563   
1      deron.beier@terrifictotes.com 2022-11-03 14:20:51.563   
2  jeanette.erdman@terrifictotes.com 2022-11-03 14:20:51.563   

             last_updated  
0 2022-11-03 14:20:51.563  
1 2022-11-03 14:20:51.563  
2 2022-11-03 14:20:51.563  
   department_id department_name    location         manager  \
0              1           Sales  Manchester    Richard Roma   
1              2      Purchasing  Manchester  Naomi Lapaglia   
2              3      Production       Leeds    Chester Ming   

               created_at            last_updated  
0 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962  
1 2022-11-03 14:20:49.962 2022-11-03 14:20:49.962  
2 2022-11-03 

In [26]:
dim_counterparty_df = dim_staff_dataframe(staff=staff_df, department=department_df)
pprint(dim_counterparty_df)

    staff_id first_name  last_name department_name    location  \
0          1    Jeremie     Franey      Purchasing  Manchester   
1          2      Deron      Beier      Facilities  Manchester   
2          3   Jeanette     Erdman      Facilities  Manchester   
3          4        Ana     Glover      Production       Leeds   
4          5  Magdalena      Zieme              HR       Leeds   
5          6      Korey    Kreiger      Production       Leeds   
6          7    Raphael     Rippin      Purchasing  Manchester   
7          8    Oswaldo  Bergstrom  Communications       Leeds   
8          9      Brody      Ratke      Purchasing  Manchester   
9         10     Jazmyn       Kuhn      Purchasing  Manchester   
10        11       Meda     Cremin         Finance  Manchester   
11        12      Imani     Walker         Finance  Manchester   
12        13       Stan     Lehner        Dispatch        Leds   
13        14  Rigoberto  VonRueden  Communications       Leeds   
14        